# Get the geohashes corresponding to Switzerland

In [199]:
import numpy as np
import geohash
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
import os
import csv
import helpers.sizeof as sizeof

In [266]:
level = 8 # Let's do this at a very high level to start filtering out all we can

In [267]:
# Get the borders of switzerland
ch = gpd.read_file("./data/switzerland.geojson")

In [268]:
# Define a function to load or generate a list of points inside the boundaries of switzerland
def get_swiss_geohashes(
        load = True,
        from_previous_level = False,
        gran = 1000, # The granularity of the grid
        level = 7, # The level of the geohashes
        filename = "geohashes_ch"):
    if from_previous_level and level > 1 and os.path.exists(f'./data/{filename}_{level-1}.csv'):
        print(f"Loading geohashes from previous level ({level-1})")
        
        with open(f'./data/{filename}_{level-1}.csv', 'r') as f:
            read = csv.reader(f)
            geohashes = list(read)

        # Get rid of nested lists
        geohashes = [gh[0] for gh in geohashes]

        # List of all possible characters in a geohash
        chars = "0123456789bcdefghjkmnpqrstuvwxyz"

        # Generate the next level of geohashes
        new_geohashes = []
        for gh in geohashes:
            for char in chars:
                new_geohashes.append(gh + char)

        # Save the new geohashes to a csv file
        ng = [[gh] for gh in new_geohashes]

        # Save the geohashes to a csv file
        with open(f'./data/{filename}_{level}.csv', 'w') as f:
            write = csv.writer(f)
            write.writerows(ng)

        geohashes = [gh[0] for gh in ng]

        print(f"Loaded {len(geohashes)} geohashes")

        return new_geohashes
    
    elif load and os.path.exists(f'./data/{filename}_{level}.csv'):
        print("Loading geohashes from file")
        
        with open(f'./data/{filename}_{level}.csv', 'r') as f:
            read = csv.reader(f)
            geohashes = list(read)

        # Get rid of nested lists
        geohashes = [gh[0] for gh in geohashes]

        print(f"Loaded {len(geohashes)} geohashes")
        
        return geohashes
    else:
        print("Generating geohashes")
        # Define the bounding box for Switzerland (from https://giswiki.hsr.ch/Bounding_Box)
        min_lat, max_lat = 45.6755, 47.9163
        min_lon, max_lon = 5.7349, 10.6677

        # Create a list of points inside the bounding box (TODO: Do this with some range function)
        # Create a grid of latitudes and longitudes
        latitudes = np.arange(int(min_lat*gran), int(max_lat*gran)+1) / gran
        longitudes = np.arange(int(min_lon*gran), int(max_lon*gran)+1) / gran

        # Create a meshgrid of latitudes and longitudes
        lon_mesh, lat_mesh = np.meshgrid(longitudes, latitudes)

        # Create a flattened array of points
        points_t = np.column_stack((lon_mesh.ravel(), lat_mesh.ravel()))

        # Convert the flattened array to a list of Shapely Points
        points_t = [Point(lon, lat) for lon, lat in points_t]

        points = gpd.GeoDataFrame(geometry=points_t, crs='EPSG:4326')

        # From points, keep only those inside the borders of Switzerland
        filtered = gpd.sjoin(points, ch, predicate='intersects')
        filtered.drop(columns=['index_right'], inplace=True)
        filtered.reset_index(drop=True, inplace=True)

        # Transform the retrieved points to geohashes
        geohashes = filtered.geometry.apply(lambda x: geohash.encode(x.y, x.x, precision=level))

        print("Before removing duplicates:", len(geohashes))

        # Get unique geohashes
        geohashes = list(set(geohashes))

        print("After removing duplicates:", len(geohashes))

        geohashes = [[gh] for gh in geohashes]

        # Save the geohashes to a csv file
        with open(f'./data/{filename}_{level}.csv', 'w') as f:
            write = csv.writer(f)
            write.writerows(geohashes)

        geohashes = [gh[0] for gh in geohashes]

        return geohashes

In [269]:
geohashes = get_swiss_geohashes(load=True, gran = 100, level=level, from_previous_level=True)

Loading geohashes from previous level (7)
Loaded 9806176 geohashes


In [270]:
sizeof.variables(locals().items())

                     geohashes: 75.5 MiB
                         rects: 65.2 MiB
                          mots: 39.4 MiB
           geohashes_filtered1: 38.3 MiB
            geohashes_filtered: 36.6 MiB
                rects_filtered: 23.4 MiB
                           _83: 18.8 MiB
                    geometries:  8.1 MiB
                          _241:  3.6 MiB
                           _62:  2.0 MiB


In [271]:
# Load the geojson with the means of transport of switzerland
mots = gpd.read_file("./results/swiss_map_mot_type.geojson")

In [272]:
def geohash_to_rect(gh):
    bbox = geohash.bbox(gh)
    return Polygon(
        [(bbox['w'], bbox['s']), (bbox['w'], bbox['n']), (bbox['e'], bbox['n']), (bbox['e'], bbox['s'])]
    )

In [273]:
# Transform all geohashes to rectangles
geometries = [geohash_to_rect(gh) for gh in geohashes]

# Create a GeoDataFrame directly from the list of geometries
rects = gpd.GeoDataFrame(geometry=geometries, crs='EPSG:4326')

rects['geohash'] = geohashes


In [274]:
len(rects)

9806176

In [275]:
# Keep only geohashes partially inside the borders of Switzerland
rects = gpd.sjoin(rects, ch, predicate='intersects')
rects.drop(columns=['index_right'], inplace=True)
rects.reset_index(drop=True, inplace=True)

In [276]:
len(rects)

9779294

In [277]:
# Keep the rectangles that intersect with at least one of the means of transport, including lakes
geohashes_filtered = gpd.sjoin(rects, mots, predicate='intersects')
geohashes_filtered.drop(columns=['index_right'], inplace=True)
geohashes_filtered.reset_index(drop=True, inplace=True)

In [278]:
def merge_dicts(x):
    return {k: v for d in x.dropna() for k, v in d.items()}

In [279]:
# Create a list with the strings of the means of transport, removing duplicates
def merge_strings(x):
    return ','.join(x.dropna().unique())

In [280]:
# Merge the geohashes with the same code, putting together all the elements in the tags column
geohashes_filtered = geohashes_filtered.groupby('geohash').agg({'type': merge_strings}).reset_index()

In [281]:
# Show the reduction in geohashes
print("Before:", len(geohashes))
print("After:", len(geohashes_filtered))
print("Difference:", len(geohashes) - len(geohashes_filtered))

Before: 9806176
After: 4111093
Difference: 5695083


In [282]:
# Save the geohashes to a csv file
geohashes_filtered['geohash'].to_csv(f'./data/geohashes_ch_{level}.csv', index=False, header=False)

In [283]:
# From rects, keep only the ones with a geohash in geohashes_filtered
rects_filtered = rects[rects['geohash'].isin(geohashes_filtered['geohash'])]

In [285]:
rects_filtered.plot(figsize=(20, 20))